In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [2]:
xls = pd.ExcelFile('./EZStacking_config.ods', engine="odf")
meta_package = pd.read_excel(xls, 'meta_package')
package_source = pd.read_excel(xls, 'package_source')
package = pd.read_excel(xls, 'package')
document = pd.read_excel(xls, 'document')

In [3]:
package_source_type = 'full'
data_size = 'large'
kaggle = True

In [4]:
pd_pk_import = package_source[(package_source.package_source_type   == package_source_type) &\
                              ((package_source.package_source_kaggle == 'both') | \
                               (package_source.package_source_kaggle == kaggle))\
                             ].merge(meta_package[(meta_package.meta_package_valid == True) & \
                                     ((meta_package.meta_package_data_size == 'both') | \
                                     (meta_package.meta_package_data_size == data_size))], \
                                     left_on  = 'meta_package_index', \
                                     right_on = 'meta_package_index', \
                                     how = 'inner') \
                             [['package_source_index', 'package_source_name', \
                               'package_source_short_name', 'package_source_code']]
pd_pk_import

,package_source_index,package_source_name,package_source_short_name,package_source_code
0,NP,numpy,np,None
1,PD,pandas,pd,None
2,SNS,seaborn,sns,None
3,OS,os,None,None
4,GC,gc,None,None
5,MPLPL,matplotlib.pyplot,plt,None
6,WARN,warnings,None,warnings.filterwarnings('ignore')
7,CPUN,None,None,os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
8,RE,re,None,None
9,MATH,math,None,None


In [5]:
package_source_type = 'partial'
data_size = 'small'

In [6]:
pd_pk_srce_from = package_source[(package_source.package_source_type == package_source_type)\
                                ].merge(meta_package[(meta_package.meta_package_valid == True) & \
                                         ((meta_package.meta_package_data_size == 'both') | \
                                         (meta_package.meta_package_data_size == data_size))], \
                                        left_on  = 'meta_package_index', \
                                        right_on = 'meta_package_index', \
                                        how = 'inner') \
                                [['package_source_index', 'package_source_name', \
                                  'package_source_short_name', 'package_source_code']] 
pd_pk_srce_from

,package_source_index,package_source_name,package_source_short_name,package_source_code
0,PDAT,pandas.api.types,None,None
1,ITT,itertools,None,None
2,JL,joblib,None,None
3,SCIP,scipy,None,None
4,SKL,sklearn,None,None
5,SKLB,sklearn.base,None,None
6,SKLE,sklearn.experimental,None,None
7,SKLI,sklearn.impute,None,None
8,SKLEN,sklearn.ensemble,None,None
9,SKLIN,sklearn.inspection,None,None


In [7]:
problem = 'regression'

In [8]:
pd_pk_from = package[((package.package_problem == 'None') | (package.package_problem == problem)) \
                    ].merge(pd_pk_srce_from, \
                            left_on='package_source_index', \
                            right_on='package_source_index',\
                            how='inner') \
                     .merge(meta_package[(meta_package.meta_package_valid == True) & \
                                         ((meta_package.meta_package_data_size == 'both') | \
                                         (meta_package.meta_package_data_size == data_size))], \
                            left_on  = 'meta_package_index', \
                            right_on = 'meta_package_index', \
                            how = 'inner') \
                [['package_source_index', 'package_source_name', 'package_name']].drop_duplicates()
pd_pk_from

,package_source_index,package_source_name,package_name
0,SKLEN,sklearn.ensemble,BaggingClassifier
5,SKLEN,sklearn.ensemble,BaggingRegressor
10,SKLEN,sklearn.ensemble,RandomForestRegressor
13,SKLEN,sklearn.ensemble,AdaBoostRegressor
14,SKLEN,sklearn.ensemble,HistGradientBoostingRegressor
15,SKLEN,sklearn.ensemble,StackingRegressor
16,SKLB,sklearn.base,BaseEstimator
17,SKLB,sklearn.base,TransformerMixin
18,SKLNE,sklearn.neighbors,LocalOutlierFactor
19,SKLE,sklearn.experimental,enable_iterative_imputer


In [9]:
pd_level_0 = package[(package.package_type == 'estimator') & \
                     (package.package_problem == problem) & \
                     (package.meta_package_index != 'STACK') \
                    ].merge(pd_pk_from, \
                            left_on='package_source_index', \
                            right_on='package_source_index',\
                            how='inner') \
                     .merge(meta_package[(meta_package.meta_package_valid == True) & \
                                         ((meta_package.meta_package_data_size == 'both') | \
                                         (meta_package.meta_package_data_size == data_size))], \
                            left_on  = 'meta_package_index', \
                            right_on = 'meta_package_index', \
                            how = 'inner') \
                    [[ 'package_index', 'package_code', 'meta_package_tree']].drop_duplicates()
pd_level_0

,package_index,package_code,meta_package_tree
0,BAGC,"BaggingClassifier(n_estimators=100, random_sta...",0.0
6,BAGCML,"BaggingClassifier(estimator=MLPClassifier(), n...",0.0
12,BAGCSVL,BaggingClassifier(estimator=SVC(kernel='linear...,0.0
18,BAGCSVP,BaggingClassifier(estimator=SVC(kernel='poly')...,0.0
24,BAGCSVR,"BaggingClassifier(estimator=SVC(kernel='rbf'),...",0.0
30,BAGR,"BaggingRegressor(n_estimators=100, random_stat...",0.0
36,BAGRML,"BaggingRegressor(estimator=MLPRegressor(), n_e...",0.0
42,BAGRSVL,BaggingRegressor(estimator=SVR(kernel='linear'...,0.0
48,BAGRSVP,"BaggingRegressor(estimator=SVR(kernel='poly'),...",0.0
54,BAGRSVR,"BaggingRegressor(estimator=SVR(kernel='rbf'), ...",0.0


In [10]:
pd_tree = package[(package.package_type == 'estimator') & \
                  (package.package_problem == problem) & \
                  (package.meta_package_index != 'STACK') \
                 ].merge(pd_pk_from, \
                         left_on='package_source_index', \
                         right_on='package_source_index',\
                         how='inner') \
                  .merge(meta_package[(meta_package.meta_package_tree == True) & \
                                      (meta_package.meta_package_valid == True) & \
                                      ((meta_package.meta_package_data_size == 'both') | \
                                      (meta_package.meta_package_data_size == data_size))], \
                         left_on  = 'meta_package_index', \
                         right_on = 'meta_package_index', \
                         how = 'inner') \
                  [[ 'package_index', 'package_code']].drop_duplicates()
pd_tree

,package_index,package_code
0,RFRS,RandomForestRegressor(criterion='squared_error...
6,RFRA,RandomForestRegressor(criterion='absolute_erro...
12,RFRP,"RandomForestRegressor(criterion='poisson', n_e..."
18,ABR,AdaBoostRegressor(random_state = random_state)
24,HGBR,HistGradientBoostingRegressor(early_stopping=T...
30,DTRF,DecisionTreeRegressor(criterion='friedman_mse'...
31,DTRA,DecisionTreeRegressor(criterion='absolute_erro...
32,DTRP,"DecisionTreeRegressor(criterion='poisson', ran..."
33,GBR,"GradientBoostingRegressor(n_estimators=100, le..."


In [11]:
pd_document = document[((document.document_problem == 'both') | (document.document_problem == problem)) & \
                       ((document.document_data_size == 'both') | (document.document_data_size == data_size)) & \
                       ((document.document_stacking == 'both') | \
                       (document.document_stacking == \
                        meta_package[meta_package.meta_package_index == 'STACK']\
                        ['meta_package_valid'].tolist()[0]\
                       )) & \
                       ((document.document_keras == 'both') | \
                       (document.document_keras == \
                        meta_package[meta_package.meta_package_index == 'KER']\
                        ['meta_package_valid'].tolist()[0]\
                       )) & \
                       ((document.document_pipeline == 'both') | \
                       (document.document_pipeline == \
                        meta_package[meta_package.meta_package_index == 'PIP']\
                        ['meta_package_valid'].tolist()[0]\
                       )) & \
                       ((document.document_yb == 'both') | \
                       (document.document_yb == \
                        meta_package[meta_package.meta_package_index == 'YB']\
                        ['meta_package_valid'].tolist()[0]\
                       )) \
                      ].merge(meta_package[meta_package.meta_package_valid == True], \
                                           left_on  = 'meta_package_index', \
                                           right_on = 'meta_package_index', \
                                           how = 'inner') \
                      [['sort_key', 'title', 'text', 'code']].sort_values('sort_key') \
                      [['title', 'text', 'code']].drop_duplicates()

pd_document

,title,text,code
0,#,EDA & Modelling,"""random_state = "" + str(random_state)"
1,##,Project name,"""name = '"" + str(project_name) + ""'"""
2,#,Exploratory Data Analysis,None
3,##,File and parameters loading,"""problem_type = '"" + str(problem_type) + ""'"""
4,,None,"""data_size = '"" + str(data_size) + ""'"""
45,,None,"""path = '"" + str(file) + ""'"""
46,,None,"""df = pd.read_csv(path)"""
5,,None,"""target_col = '"" + str(target_col) + ""'"""
6,###,Thresholds & other parameters,"""threshold_NaN = "" + str(threshold_NaN)"
7,,None,"""threshold_cat = "" + str(threshold_cat)"
